# Plot scoring positions

Use model to inference a new map



In [ ]:
#import fnmatch
#import os
#import pathlib
#import random
#import time

import intel_extension_for_pytorch as ipex
import matplotlib.pyplot as plt
import numpy as np
#import psutil
import seaborn as sns
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
#from tqdm import tqdm
from PIL import Image
from data_loader import (
    TRAIN_DIR,
    VALID_DIR,
    augment_and_save,
    data_distribution,
    imagenet_stats,
    img_transforms,
    plot_data_distribution,
    show_data,
)
sns.set_theme(style="whitegrid", palette="husl")

# Specify Model to test

In [ ]:
# simple = False
# epochs = 12
# ipx = True 
# dropout = .5
# batch_size = 128
# # bc_resnet18_simpleFalse_IPEXTrue_Epochs12_dropout0.4_batch128
# model_name = "ON_resnet18_simpleFalse_IPEXTrue_Epochs40_dropout0.33_batch128"
# #resnet18_simpleFalse_IPEXTrue_Epochs12_dropout0.4_batch128
# #model_name = f"resnet18_simple{simple}_IPEX{ipx}_Epochs{epochs}_dropout{dropout}_batch{batch_size}"
# print(model_name)
# model_read = torch.jit.load(f"models/{model_name}.pt")

# model_read.eval()

# Define Predict Function

In [ ]:
def predict(InFldr, ScoreDict, model):
    import torchvision.transforms as transforms
    import torch.nn as nn
    import glob 
    model.eval()

    scores = []
    fns = []
    transform = transforms.Compose([
        transforms.Resize((1024, 1024)),
        transforms.ToTensor(),
        transforms.Normalize(*imagenet_stats),
    ])
    
    for fn in glob.glob(TEST_DIR+'*.png'):
        img = Image.open(fn)
        img_tensor = transform(img).reshape(1, 3, 1024, 1024)
        score = 0
        modelCPU = model.to("cpu")
        score = np.argmax(torch.softmax( modelCPU(img_tensor).detach(), dim =1).numpy().squeeze())
        fns.append(fn)
        scores.append(ScoreDict[score])
    return (scores, fns)
# model.eval()
# with torch.no_grad():
#     model_scripted = torch.jit.script(model.to("cpu"))  # Export to TorchScript
#     model_scripted.save(model_save_path)  # Jit Save

In [ ]:
#model_save_path
model_name = f"models/model_acc_84_device_xpu_lr_0.000214_epochs_20_jit.pt"
print(model_name)

model_read = torch.jit.load(model_name)
#model_read.eval()

# Scoring individual images with model
### Known Fire

In [ ]:
ytrue = []
ypred = []
TEST_DIR =  'data/colorEnhanced/val/Fire/'
scoreDict = {0:'Fire', 1:'NoFire'}
scores, fns = predict(TEST_DIR, scoreDict, model_read)
for score, fn in zip(scores, fns):
    print(f"{score}\t{fn}")
    if score == 'Fire': 
        ypred.append(1)
    else:
        ypred.append(0)
    ytrue.append(1)

# Scoring individual images with model
### Known NoFire

# Define functions and normalization parameters

In [ ]:
TEST_DIR =  'data/colorEnhanced/val/NoFire/'
scoreDict = {0:'Fire', 1:'NoFire'}
scores, fns = predict(TEST_DIR, scoreDict, model_read)
for score, fn in zip(scores, fns):
    print(f"{score}\t{fn}")
    if score == 'Fire': 
        ypred.append(1)
    else:
        ypred.append(0)
    ytrue.append(0)

In [ ]:
%matplotlib inline
fireDict = {0:'Fire', 1:'NoFire'}
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
YTRUE = [fireDict[val] for val in ytrue]
YPRED = [fireDict[val] for val in ypred]
labels=['Fire', 'NoFire']
cm = confusion_matrix(YTRUE, YPRED,  labels=labels)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot()

plt.show()

## Notices and Disclaimers

Intel technologies may require enabled hardware, software or service activation.

No product or component can be absolutely secure. 

Your costs and results may vary. 

© Intel Corporation. Intel, the Intel logo, and other Intel marks are trademarks of Intel Corporation or its subsidiaries. Other names and brands may be claimed as the property of others. 